In [ ]:
import os
import io
import glob
import sys
import astropy.io.fits
import matplotlib
import numpy as np
import time
import timeit

import psycopg2

import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)  
import radiospectra
from radiospectra.sources import CallistoSpectrogram

from matplotlib.backends.backend_pdf import PdfPages, FigureCanvasPdf, PdfFile
import config as test_config

import warnings
warnings.filterwarnings("ignore")

In [ ]:
connection = psycopg2.connect(host= test_config.DB_HOST,
                              database= test_config.DB_DATABASE,
                              user= test_config.DB_USER,
                              port= test_config.DB_PORT, 
                              password=test_config.DB_PASSWORD)

cursor = connection.cursor()

In [ ]:
cursor.execute("""SELECT * from data WHERE std is null ORDER BY id""")

In [ ]:
for i in cursor:
    print(i)

In [ ]:
import os

path = 'R:\\radio\\2002-20yy_Callisto\\2017\\09\\06'


In [ ]:
out = os.path.basename(path)
print(out)

In [ ]:
out = os.path.dirname(path)
print(out)

In [ ]:
out = os.path.isabs(path)
print(out)

In [ ]:
out = os.path.isdir(path)
print(out)

In [ ]:
out = os.path.isfile(path)
print(out)

In [ ]:
out = os.path.normcase("/BAz")
print(out)

In [ ]:
out = os.path.normpath(path)
print(out)

In [ ]:
import sys
print(sys.path)

In [ ]:
sys.platform

In [ ]:
print(sys.stdin)

In [ ]:
print(sys.stdout)

In [ ]:
print(sys.stderr)

In [ ]:
os.getcwd()

In [ ]:
def move_axes(fig, ax_source, ax_target):
    old_fig = ax_source.figure
    ax_source.remove()
    ax_source.figure = fig
    ax_source.set_ylabel('')
    ax_source.set_xlabel('')
        
    ax_source.set_position(ax_target.get_position())
    ax_target.remove()
    ax_target.set_aspect("equal")
    fig.axes.append(ax_source)
    fig.add_subplot(ax_source)
    
    plt.close(old_fig)

# Connection between Jupyter notebook and Postgres
connection = psycopg2.connect(host= test_config.DB_HOST,
                              database= test_config.DB_DATABASE,
                              user= test_config.DB_USER,
                              port= test_config.DB_PORT, 
                              password=test_config.DB_PASSWORD)

cursor = connection.cursor()

# Choosing 10 images for each Station
cursor.execute("""select * from (
                                 select ROW_NUMBER() OVER (partition by path order by id)
                                 as row_num, data.* FROM data
                                 ) t
                                 where row_num <=10
                                 order by id""")


my_colormap = matplotlib.colors.LinearSegmentedColormap.from_list("myColorMap", plot_config.COLORMAP / 255)

count = 0

# with PdfPages('Bg_Sub_Images.pdf') as pdf:

for file in cursor.fetchall():
    if count < 5 :

        spec = CallistoSpectrogram.read(test_config.DATA_PATH + file[2])
        fig1, axs1 = plt.subplots(1, 4, figsize=(20, 5))
        ax1 = spec.plot()
        ax1.title.set_text("Original Data")
        plt.close()

        # Second column, Constbacksub + elimwrongchannels
        spec2 = spec.subtract_bg("constbacksub", "elimwrongchannels")
        fig2 = plt.subplots(1, 4, figsize=(20, 5))
        ax2 = spec2.plot(vmin=-5, vmax=5)
        ax2.title.set_text("Background subtracted")
        plt.close()

        # Third column, subtract_bg_sliding_window
        spec3 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                                 amount=0.05, change_points=True)
        fig3 = plt.figure(figsize=(20, 5))
        ax3 = spec3.plot(vmin=-5, vmax=5)
        ax3.title.set_text("Gliding background subtracted (window=800)")
        plt.close()
        
#         std = spec2.data.std()
#         mean = spec2.data.mean()

        # Fourth column, Histograms
        data_hist3 = np.absolute(spec2.data.flatten())
        std = data_hist3.std()
        mean = data_hist3.mean()
        ax4.hist(data_hist3, histtype='step', range= (0, 10),bins=20, label='Background subtracted')
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mean, std)
        plt.plot(x, p, 'k', linewidth=2)
        
        
#         ax4.hist(data_hist4, histtype='step', range= (0, 10),bins=20, label='Gliding background subtracted')
        
#         ax4.title.set_text("Histograms")
        plt.legend()
        plt.close()

        # Plot final plot by moving axes to the figure
        fig_target, (axA, axB, axC, axD) = plt.subplots(1, 4, figsize=(30,7))
        plt.suptitle(fig1._suptitle.get_text())

        move_axes(fig_target, ax1, axA)
        move_axes(fig_target, ax2, axB)
        move_axes(fig_target, ax3, axC)
        move_axes(fig_target, ax4, axD)

        ax1.set_xlabel('Time[UT]')
        ax1.set_ylabel('Frequency[MHz]')

        ax2.set_xlabel('Time[UT]')
        ax2.set_ylabel('Frequency[MHz]')

        ax3.set_xlabel('Time[UT]')
        ax3.set_ylabel('Frequency[MHz]')

        ax4.set_xlabel('Pixel value')
        ax4.set_ylabel('Number of pixels')

        plt.show()

#         count += 1
#         pdf.savefig(fig_target)
        plt.close()


print("Finished plotting!")


In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# Generate some data for this
# demonstration.
data = np.random.normal(170, 10, 250)

# Fit a normal distribution to
# the data:
# mean and standard deviation
mu, std = norm.fit(data)

# Plot the histogram.
plt.hist(data, bins=25, density=True, alpha=0.6, color='b')

# Plot the PDF.
xmin, xmax = plt.xlim()
print(xmin)
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)

plt.plot(x, p, 'k', linewidth=2)
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)

plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt
x = random.sample(range(1000), 100)
xbins = [0, len(x)]
plt.hist(x, bins=xbins, color = 'blue') 
#Does not make the histogram correct. It counts the occurances of the individual counts. 

plt.plot(x)
#plot works but I need this in histogram format


In [ ]:
from matplotlib import pyplot as plt 
import numpy as np  
   
a = np.array([22,87,5,43,56,73,55,54,11,20,51,5,79,31,27]) 
plt.hist(a, bins = 'auto') 
plt.title("histogram") 
plt.show()

In [1]:
import os
import io
import glob
import sys
import astropy.io.fits
import matplotlib
import numpy as np
import time
import timeit
import skimage.transform

import psycopg2
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import re 
from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import radiospectra

from radiospectra.sources import CallistoSpectrogram

from matplotlib.backends.backend_pdf import PdfPages, FigureCanvasPdf, PdfFile
import datetime
import warnings
warnings.filterwarnings("ignore")

# print("Hello")

                



C:\Users\delbe\OneDrive\Desktop\eCallistoValidation\radiospectra\radiospectra\version.py:12: UserWarning: could not determine radiospectra package version; this indicates a broken installation
  warnings.warn(


In [10]:
def __to_timestamp(date_string, time_string):

    sixty_seconds = int(time_string[6:8]) == 60
    sixty_minutes = int(time_string[3:5]) == 60
    twentyfour_hours = int(time_string[:2]) == 24

    # replacing  24 to 00
    if sixty_seconds:
        time_string = time_string[:6] + '59' + time_string[8:]
    if sixty_minutes:
        time_string = time_string[:3] + '59' + time_string[5:]
    if twentyfour_hours:
        time_string = '23' + time_string[2:]
    if re.findall("\.\d+", time_string):
        time_string = time_string[:-4]

    # lost time
    ts = datetime.datetime.strptime(
        '%s %s' % (date_string, time_string), '%Y/%m/%d %H:%M:%S')
    ts += datetime.timedelta(hours=int(twentyfour_hours),
                             minutes=int(sixty_minutes),
                             seconds=int(sixty_seconds))

    return ts

In [12]:
print("Hello")
path = '/data/radio/2002-20yy_Callisto/2017/09'
print(path)
engine = create_engine("postgresql+psycopg2://" + 'postgres' + ":" + 'ecallistohackorange' + "@" + 'localhost' + "/" + 'validation')
# print(engine)
df = 0

for root, dirs, files in os.walk(path):
    print("path")
    for name in files:
        if name.endswith('.fit.gz'):
            full_path = os.path.join(root, name)
           # print("full_path")
            
            hdulist = astropy.io.fits.open(full_path)
            split_path = full_path.split("Callisto")
            print("split_path")


            instrument_name = hdulist[0].header['INSTRUME'] 
            date_obs = hdulist[0].header['DATE-OBS'] 
            time_obs = hdulist[0].header['TIME-OBS']
            date_end = hdulist[0].header['DATE-END'] 
            time_end = hdulist[0].header['TIME-END'] 
            
            # combine date and time obs, date and time end
            start_time = __to_timestamp(date_obs, time_obs)
            end_time = __to_timestamp(date_end, time_end)
            
            # creating dataframe in pandas
            
            data = {
                'path': [split_path[1]],
                'file_name': [name],
                'instrument_name': [instrument_name],
                'start_time': [start_time],
                'end_time': [end_time],
                'std': [None]
                  }
             
            data_frame = pd.DataFrame(data, index=[df])
            
            # connection between pandas and sql 
            data_frame.to_sql('validation_data', con=engine, if_exists='append', chunksize=10000000, index=False)
                       
            df = df + 1
                

Hello
/data/radio/2002-20yy_Callisto/2017/09
